In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML 

import re
import os
from tqdm import tqdm
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import NormalPredictor, KNNBasic, SVD, SVDpp

In [2]:
from recommender_metrics import RecommenderMetrics
from movie_lens_data import MovieLensData
from evaluator import Evaluator

In [3]:
path = "./data/ml-100k"
movie_lens_data = MovieLensData(
    users_path = os.path.join(path, "u.user"),
    ratings_path = os.path.join(path, "u.data_new0"),
    movies_path = os.path.join(path, "u.item"), 
    genre_path = os.path.join(path, "u.genre") 
    )

evaluation_data = movie_lens_data.read_ratings_data()
movie_data = movie_lens_data.read_movies_data()
popularity_rankings = movie_lens_data.get_popularity_ranks()
ratings = movie_lens_data.get_ratings()

In [4]:
evaluator = Evaluator(evaluation_data, popularity_rankings)

Number of full trainset users: 935
Number of full trainset items: 1645
Number of trainset users: 935
Number of trainset items: 1645
Size of testset: 79275
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


## ItemKNN

In [5]:
item_KNN = KNNBasic(sim_options = {'name': 'cosine', 'user_based': False})
evaluator.add_algorithm(item_KNN, "Item KNN")

## UserKnn

In [7]:
user_KNN = KNNBasic(sim_options = {'name': 'pearson', 'user_based': True})
evaluator.add_algorithm(user_KNN, "User KNN")

## SVD

In [8]:
SVD = SVD()
evaluator.add_algorithm(SVD, "SVD")

## SVD++

In [11]:
SVD_plus_plus = SVDpp()
evaluator.add_algorithm(SVD_plus_plus, "SVD++")

## Random

In [12]:
# adding random evaluator
algo_np = NormalPredictor()
evaluator.add_algorithm(algo_np, "Random")

In [9]:
evaluator.evaluate(do_top_n=False)

Evaluating  Item KNN ...
Evaluating accuracy...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  User KNN ...
Evaluating accuracy...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD ...
Evaluating accuracy...
Analysis complete.


Algorithm  RMSE       MAE        FCP       
Item KNN   0.8899     0.6987     0.8269    
User KNN   0.7556     0.5992     0.8688    
SVD        0.6833     0.5419     0.8820    

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
FCP:       Fraction of Concordant Pairs. Higher values mean better accuracy.


In [14]:
# Time consuming, uncomment optionally
evaluator.evaluate(do_top_n=True)

Evaluating  Item KNN ...
Evaluating accuracy...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating top-N with leave-one-out...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  User KNN ...
Evaluating accuracy...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating top-N with leave-one-out...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Analyzing coverage, diversity, and novelty...
Comput

# Evaluate topN recommendations

In [ ]:
# evaluator.sample_top_n_recs(movie_lens_data, test_subject=85, k=10)

In [ ]:
# evaluator.sample_top_n_recs(movie_lens_data, test_subject=314, k=5)

In [ ]:
# print(movie_lens_data.get_movie_name(44))
# evaluator.sample_top_n_recs(movie_lens_data, test_subject=44, k=10)

In [ ]:
# list_of_recommendations=evaluator.sample_top_n_recs(movie_lens_data, test_subject=1, k=10)
# print(list_of_recommendations)

# Evaluation

In [ ]:
import numpy as np

def ndcg_score(y_true, y_pred, k=None):
    # Calculate the Normalized Discounted Cumulative Gain (NDCG) score.
    # Convert inputs to numpy arrays
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    # Ensure y_true and y_pred have the same shape
    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")
    
    # Sort indices of y_pred in descending order
    sorted_indices = np.argsort(y_pred)[::-1]
    
    # Select top-K elements if k is specified
    if k is not None:
        sorted_indices = sorted_indices[:k]
    
    # Calculate DCG
    dcg = np.sum((2 ** y_true[sorted_indices] - 1) / np.log2(np.arange(2, len(sorted_indices) + 2)))
    
    # Sort y_true in descending order and calculate ideal DCG
    ideal_sorted_indices = np.argsort(y_true)[::-1]
    ideal_dcg = np.sum((2 ** y_true[ideal_sorted_indices] - 1) / np.log2(np.arange(2, len(ideal_sorted_indices) + 2)))
    
    # Calculate NDCG
    ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0
    return ndcg

In [ ]:
def average_precision(y_true, y_pred, k=None):
    """
    Calculate the Average Precision (AP) score for a single query.
    """
    if k is not None:
        y_true = y_true[:k]
        y_pred = y_pred[:k]
    
    # Initialize variables
    precision_sum = 0.0
    true_positives = 0
    
    # Calculate precision at each relevant document
    for i, (true, pred) in enumerate(zip(y_true, y_pred), 1):
        if true:
            true_positives += 1
            precision = true_positives / i
            precision_sum += precision
    
    # Calculate Average Precision
    if true_positives == 0:
        return 0.0
    else:
        return precision_sum / true_positives

In [ ]:

def mean_average_precision(y_true_list, y_pred_list, k=None):
    """
    Calculate the Mean Average Precision (MAP) score for a list of queries.
    
    Parameters:
    y_true_list (list of arrays): List of true relevance scores for each query.
    y_pred_list (list of arrays): List of predicted relevance scores for each query.
    
    Returns:
    float: The Mean Average Precision (MAP) score.
    """
    total_queries = len(y_true_list)
    sum_ap = 0.0
    
    for y_true, y_pred in zip(y_true_list, y_pred_list):
        sum_ap += average_precision(y_true, y_pred, k)
    
    # Calculate MAP
    map_score = sum_ap / total_queries
    return map_score



In [ ]:
def mean_rating_deviation(y_true, y_pred, k=None):
    if k is not None:
        y_true = y_true[:k]
        y_pred = y_pred[:k]
    
    # Convert inputs to numpy arrays
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    # Ensure y_true and y_pred have the same shape
    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")
    
    # Calculate Mean Rating
    mrd_score = np.mean(y_pred)-np.mean(y_true)
    return mrd_score

In [ ]:
def mean_reciprocal_rank(y_true_list, y_pred_list, k=None):

    if k is not None:
        y_true = y_true[:k]
        y_pred = y_pred[:k]
    
    total_users = len(y_true_list)
    sum_rr = 0.0

    for y_true, y_pred in zip(y_true_list, y_pred_list):
        # Sort predicted ratings and corresponding true ratings
        sorted_items = sorted(range(len(y_pred)), key=lambda i: y_pred[i], reverse=True)
        true_sorted = [y_true[i] for i in sorted_items]

        # Find the rank of the first relevant item
        first_relevant_rank = next((i + 1 for i, rating in enumerate(true_sorted) if rating), None)

        # Calculate reciprocal rank
        reciprocal_rank = 1 / first_relevant_rank if first_relevant_rank else 0

        sum_rr += reciprocal_rank

    # Calculate MRR
    mrr_score = sum_rr / total_users
    return mrr_score


In [ ]:
def calc_score(algo_dict, userid, y_true, y_pred, k=None):
    ndcg = ndcg_score(y_true, y_pred,k)
    # print("NDCG score:", ndcg)
    ap=average_precision(y_true, y_pred,k)
    # print('Average Precision', ap)
    mrd_score = mean_rating_deviation(y_true, y_pred,k)
    # print("Mean Rating (MR) deviation score:", mrd_score)

    if userid in algo_dict:
        # If the key exists
        algo_dict[userid].append([ndcg, ap, mrd_score])
    else:
        # If the key does not exist
        algo_dict[userid]=([ndcg, ap, mrd_score])
    return algo_dict

## Evaluation from generated data

Merging the ratings pridicted by 5 algorithms from Selected_{userid}_ {algo} for each userid in RecSys_{userid}.csv

<b>selected_userid_algo.csv</b> files: contains the the calculated score for that user for movies with given ground truth rating based on selected algorithm and it is used to evaluate.

In [ ]:
def dataGen(userid):
    path="./Data"
    algo=0
    file_path=os.path.join(path, f'selected{userid}_{algo}.csv')
    df=pd.read_csv(file_path,)
    os.remove(file_path)
    
    algo+=1
    file_path=os.path.join(path, f'selected{userid}_{algo}.csv')
    df_1=pd.read_csv(file_path,)
    os.remove(file_path)
    
    algo+=1
    file_path=os.path.join(path, f'selected{userid}_{algo}.csv')
    df_2=pd.read_csv(file_path,)
    os.remove(file_path)
    
    algo+=1
    file_path=os.path.join(path, f'selected{userid}_{algo}.csv')
    df_3=pd.read_csv(file_path,)
    os.remove(file_path)
    
    algo+=1
    file_path=os.path.join(path, f'selected{userid}_{algo}.csv')
    df_4=pd.read_csv(file_path,)
    os.remove(file_path)

    df = df.assign(UserKNN=df_1['est'])
    df = df.assign(SVD=df_2['est'])
    df = df.assign(SVDpp=df_3['est'])
    df = df.assign(Random=df_4['est'])
    df = df.rename(columns={'est':'ItemKnn','SVDpp':'SVD++'})
    df.to_csv(f'.//Results//RecSys_{userid}.csv', index=False)

Using our test data for ground-truth ratings

In [ ]:
file_path = "D:\\Projects\\P_DSc\\Intern-IITP\\NB\\data\\ml-100k\\u.data.test"  # Specify the path to your TSV file
test_data = pd.read_csv(file_path, sep='\t')

order=['userid', 'itemid', 'rating', 'timestamp']
test_data.columns= order

def sort_column_within_group(group):
    group=group.sort_values(by='rating', ascending=False)
    return group
test_data = test_data.groupby('userid').apply(sort_column_within_group)
test_data = test_data.reset_index(drop=True)
print(test_data)

       userid  itemid  rating  timestamp
0           1      15       5  875071608
1           1      93       5  875071484
2           1     253       5  874965970
3           1     223       5  876892918
4           1     216       5  876892701
...       ...     ...     ...        ...
20277     943     419       2  888638920
20278     943      97       2  888639445
20279     943     720       1  888640048
20280     943     941       1  888639725
20281     943     399       1  888639886

[20282 rows x 4 columns]


In [ ]:
score_dict={
    'ItemKNN':{},
    'UserKNN':{},
    'SVD':{},
    'SVD++':{},
    'Random':{},
}
score_dict_5={
    'ItemKNN':{},
    'UserKNN':{},
    'SVD':{},
    'SVD++':{},
    'Random':{},
    'userid':{},
}

userids = set(test_data['userid'])
# userids1 = set(test_data['userid'])
# intersection = set1 & set2

for userid in tqdm(userids):
    
    list_of_recommendations=evaluator.sample_top_n_recs(movie_lens_data, test_subject=userid, k=10)
    
    test_i_user = test_data[test_data['userid'].isin([userid])].drop(columns=['userid', 'timestamp']).sort_values(by='itemid', ascending=True).reset_index(drop=True)
    # test_i_user.to_csv(f'.//Results//test_{userid}.csv')
    # print(test_i_user)
    print(f'\n{userid}\n')
    score_dict

    for i in range(5):
        order=['itemid', 'act', 'est']
        prediction_df=pd.DataFrame(list_of_recommendations[i], columns=order).sort_values(by='itemid', ascending=True).reset_index(drop=True)
        
        selected_items = prediction_df[prediction_df['itemid'].isin(test_i_user['itemid'])]
        selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values
        selected_items=selected_items.sort_values(by='est', ascending=False)
        # display(selected_items)
        selected_items.to_csv(f'.//Results//selected{userid}_{i}.csv', index=False)

        k=None
        
        y_true = selected_items['act'].values
        y_pred = selected_items['est'].values
        
        algo=('ItemKNN') if (i==0) else ('UserKNN') if(i==1) else ('SVD') if (i==2) else ('SVD++') if (i==3) else ('Random') if(i==4) else ('algo')
        algo_dict=score_dict[algo]
        algo_dict=calc_score(algo_dict, userid, y_true, y_pred, k)
        
        algo_dict_5=score_dict_5[algo]
        algo_dict_5=calc_score(algo_dict_5, userid, y_true, y_pred, k=5)
    dataGen(userid)

#### Evaluation old metric functions

In [ ]:
# # generate list of (list of act and est) for each user
# def read_csv_and_append_lists(csv_file, y_true_list, y_pred_list, algo):
#     # Read CSV into DataFrame
#     df = pd.read_csv(csv_file)
#     true_list=[]
#     pred_list=[]
#     for index, row in df.iterrows():
#         true_list.append(row[df.columns[1]])
#         pred_list.append(row[df.columns[algo+2]])
    
#     y_true_list.append(true_list)
#     y_pred_list.append(pred_list)
#     return y_true_list, y_pred_list

In [ ]:
# def map_mrr_eva(df, k=None):
#     map_list=['Total_map']
#     mrr_list=['Total_mrr']
#     for algo in range(5):
#         y_true_list=[]
#         y_pred_list=[]
#         for userid in tqdm(userids):
#             read_csv_and_append_lists(f'.//Data//RecSys_{userid}.csv', y_true_list, y_pred_list, algo)
#             # read recsys_{userid}.csv file in df append the y_true_list with col[0] and y_pred_list with column[algo+1]
#         map_list.append(mean_average_precision(y_true_list,y_pred_list,k))
#         mrr_list.append(mean_reciprocal_rank(y_true_list,y_pred_list,k))

#     # df.loc[len(df.index)] = map_list
#     print(map_list)
#     print(mrr_list)
#     # df.loc[len(df.index)] = mrr_list
#     # return df

In [ ]:
# # map_list=['tat',1,1,1,1,1]
# # map_df=pd.DataFrame([[1,1,1,1,1,1]],columns=['userid', 'ItemKNN', 'UserKNN', 'SVD', 'SVD++', 'Random'])
# # map_df.loc[len(df.index)] = map_list
# score_df = map_mrr_eva(score_df, k=None)
# display(score_df)

100%|██████████| 935/935 [00:01<00:00, 554.59it/s]

['Total_map', 1.0, 1.0, 1.0, 1.0, 1.0]
['Total_mrr', 1.0, 1.0, 1.0, 1.0, 1.0]


,userid,ItemKNN,UserKNN,SVD,SVD++,Random
1,1,"[0.913833377284391, 1.0, 0.028218400445717773]","[0.903913221476052, 1.0, -0.03201403323678775]","[0.883227526388415, 1.0, 0.057193770345039585]","[0.8839403603743479, 1.0, -0.03217240607797667]","[0.8211583945321769, 1.0, -0.19606327524910938]"
2,2,"[0.7876114186959077, 1.0, 0.03886367782789213]","[0.9290354062872561, 1.0, -0.028514534166727667]","[0.9304093393052109, 1.0, -0.058486899471716924]","[0.968565215805775, 1.0, 0.12118752850663128]","[0.9261133219164371, 1.0, -0.08584599087559619]"
3,3,"[0.6301432645196707, 1.0, 0.12180420158433947]","[0.7507555555602629, 1.0, 0.6103075219295526]","[0.7175993560461386, 1.0, 0.10086121570028883]","[0.7390208704910455, 1.0, 0.038999739348986484]","[0.6379201487463476, 1.0, 0.6755159597811504]"
4,4,"[0.9953395684454479, 1.0, 0.0348836678265565]","[0.921310321485301, 1.0, -1.039002271660347]","[0.9639021258532504, 1.0, -0.3675108050550273]","[0.9953395684454479, 1.0, -0.2129825046617153]","[0.9073866124623576, 1.0, -0.758871402672388]"
5,5,"[0.9018010021736702, 1.0, -0.07362574691777901]","[0.9263369142592415, 1.0, 0.4140071713037332]","[0.8391843772352486, 1.0, -0.016783820969987318]","[0.9547372212458504, 1.0, 0.0029755187678688344]","[0.7313587319307674, 1.0, 0.41029879335633]"
...,...,...,...,...,...,...
939,939,"[0.8486582655649633, 1.0, -0.02914841451013217]","[0.9838939725958612, 1.0, -1.1690702644453963]","[0.9995843660616693, 1.0, -0.12424925976173462]","[0.980813297963661, 1.0, -0.12377103385161004]","[0.8880534438167289, 1.0, -0.4563167135369177]"
940,940,"[0.7775411711141598, 1.0, -0.015473788107752462]","[0.9029190085899952, 1.0, 0.2811384844641647]","[0.8575570345510818, 1.0, -0.011677010067402183]","[0.925158263698964, 1.0, 0.11236576964102918]","[0.853855202652721, 1.0, -0.12068238036529255]"
941,941,"[0.8862372091245206, 1.0, -0.4539089251043582]","[0.8862372091245206, 1.0, -0.5036857033870805]","[0.9544372751968143, 1.0, -0.3450241277501176]","[0.8862372091245206, 1.0, -0.4772939950143096]","[0.8862372091245206, 1.0, -0.5750858255225881]"
942,942,"[0.907470106580751, 1.0, 0.2039601319047808]","[0.9059593591032901, 1.0, -0.28666691624051666]","[0.8316012109081095, 1.0, 0.16361012648468343]","[0.8468808116865268, 1.0, 0.1717625692413689]","[0.8934390074553678, 1.0, -0.5055084170179232]"


In [ ]:
# print(score_dict)
# score_df=pd.DataFrame(score_dict)
# score_df.insert(0, 'userid', score_df.index)
# # map_df = map_mrr_eva(score_df, k)
# score_df.to_csv(f'.//Results//score.csv', index=False)

# score_df_5=pd.DataFrame(score_dict_5)
# score_df_5.drop(columns=['userid'],axis=1,inplace=True)
# score_df_5.insert(0, 'userid', score_df_5.index)
# # score_df_5 = map_mrr_eva(score_df, k=5)
# score_df_5.to_csv(f'.//Results//score_5.csv', index=False)

## Evaluating using new metrics

In [ ]:
import numpy as np
import pandas as pd
import math

import numpy as np
import math
def order_indices(ratings, imdb_list):
    '''generate the sequece of indices as rank based on the rating and imdb rating values. First rank at first position'''
    rating_array = np.array(ratings)
    unique_ratings = sorted(list(set(ratings)))[::-1]

    order_indics = []

    for ur in unique_ratings:
        rtng_indices = np.where(rating_array == ur)[0]
       

        imdb = [imdb_list[r_indx] for r_indx in rtng_indices]
        imdb = list(set(imdb))
        imdb = sorted(imdb, reverse=True)
      

        for i in imdb:
            for r_indx in rtng_indices:
                if imdb_list[r_indx] == i:
                    order_indics.append(r_indx)       

    return order_indics

def dcg_at_K(actual_ratings, indeces_order, K):
    '''generates the dcg on a cut of value of K'''
    k, dcg = 0, 0
    K = K if K<=len(actual_ratings) else len(actual_ratings)

    for indx, rating_indx in enumerate(indeces_order):
        relavance = actual_ratings[rating_indx]
        numerator = pow(2,relavance) - 1
        denomirator = math.log2(indx+2)
        dcg += numerator/denomirator

        k += 1
        if k == K:
            return dcg
        

def mrr_at_k(ac_order_indc, pr_order_indc, actual_ratings, predicted_ratings, K):
    '''generates mrr@k'''
    k, rr = 0, 0
    K = K if K<=len(predicted_ratings) else len(predicted_ratings)
    for i, indx in enumerate(pr_order_indc):
        if k<K:
            if predicted_ratings[indx] >= 3 and actual_ratings[ac_order_indc[i]] >= 3:
                rr += 1/(i+1)
            elif predicted_ratings[indx] < 3 and actual_ratings[ac_order_indc[i]] < 3:
                rr += 1/(i+1)
        k += 1
    MRR_at_k = rr/K
    return(MRR_at_k)

def fcp_at_K(actual_ratings, pr_order_indc, K): #Fraction of Concordant Pairs at K
    K = K if K<=len(actual_ratings) else len(actual_ratings)
    concordant = 0

    top_k_indices = pr_order_indc[:K]

    for i, indx in enumerate(top_k_indices):
        if i < (K-1):
            nxt_top_k_indices = pr_order_indc[i+1:K]
            for nxt_indics in nxt_top_k_indices:
                concordant += 1 if actual_ratings[indx] >= actual_ratings[nxt_indics] else 0

    total_pair = math.comb(K, 2)
    fcp_at_k = concordant/total_pair
    return fcp_at_k

def ndcg_mrr_fpc_k(actual_ratings, predicted_ratings, imdb_list, K):
    '''takes the actual and predicted ratings of movies with imdb list and cutoff k. Returns the ndgc@K, mrr@k and fcp@k'''
    ac_order_indc = order_indices(actual_ratings, imdb_list)
    pr_order_indc = order_indices(predicted_ratings, imdb_list)

    # calculating ndcg
    ndcg_k = dcg_at_K(actual_ratings, pr_order_indc, K)/dcg_at_K(actual_ratings, ac_order_indc, K)

    # calculating mrr
    MRR_at_k = mrr_at_k(ac_order_indc, pr_order_indc, actual_ratings, predicted_ratings, K)

    # calculating fcp
    fcp_at_k = fcp_at_K(actual_ratings, pr_order_indc, K)


    return (ndcg_k, MRR_at_k, fcp_at_k)

def MAE_MSE_RMSE_R2_at_K(actual_ratings, predicted_ratings, K):
    '''it returns MAE@K, MSE@K, RMSE@K and R2'''

    K = K if K<=len(actual_ratings) else len(actual_ratings)

    k, AE, SE, SS_res, SS_tot = 0, 0, 0, 0, 0
    ac_avg = np.average(np.array(actual_ratings))

    for indx, _ in enumerate(actual_ratings):
        if k < K:
            #  calculate AE
            AE += abs(actual_ratings[indx] - predicted_ratings[indx])

            # calculate SE
            error = (actual_ratings[indx] - predicted_ratings[indx])
            SE += pow(error, 2)


        # calculate R2
        error = (actual_ratings[indx] - predicted_ratings[indx])
        SS_res += pow(error, 2)
        error = actual_ratings[indx] - ac_avg
        SS_tot += pow(error, 2)

        k += 1
       

    MAE_at_K = AE/K
    MSE_at_K = SE/K
    RMSE_at_K = pow(MSE_at_K, 0.5)
    try:
        R2 = 1 - (SS_res/SS_tot)
    except Exception as e:
        print(e)

    return(MAE_at_K, MSE_at_K, RMSE_at_K, R2)

### Data Generator
#### --Evaluating using actual and predicted ratings from all models already saved in csv
<b>RecSys_{userid}.csv: </b>It contains the rating for a user on items from all Models(<b>here used just to extract the actual rating with itemid</b> which was extracted and formatted earlier)

-To reduce computation time(by not recalculating ratings again)

In [ ]:
# The first line of the code creates a list called score with the following lists: ['userid', 'Model Name', 'K', 'ndcg@k', 'Relative_mrr@k', 'fcp@k', 'MAE@k', 'MSE@k', 'RMSE@k', 'R2squared'].
# It then reads in a CVS file called imdb_df using the pd.read_csv() function.
# It then loops through a list of userids and for each userid, it reads in a CSV file called user_df using the pd.read_csv() function.
# It then loops through the range of 5 and values_k.
# It then calculates the actual values y_true and predicted values y_pred for each user.
# It then calculates the ndcg, mrr, fcp, MAE, MSE, RMSE and R2squared using custom functions (not provided in the code)
# It then assigns the algorithm name (e.g. 'ItemKNN') to the variable algo.
# It then appends the user id, algorithm name, K value and the results of the custom functions to the list score.
# It continues to do this for each use


# score=[['userid', 'Model Name', 'K', ['y_true'], ['y_pred'], ['y_IMDB']]]
score=[['userid', 'Model Name', 'K', 'ndcg@k', 'Relative_mrr@k','fcp@k', 'MAE@k', 'MSE@k', 'RMSE@k', 'R2squared']]
userids=score_df['userid'].tolist()
values_k=[3, 5, 10, 15, 20]
imdb_df=pd.read_csv('.//Data//ml-100k//movie_with_imdb.csv')
for userid in userids:
    user_df=pd.read_csv(f'.//data//Own//fullv0//RecSys_{userid}.csv')
    for i in range(5):
        for k in values_k:
            y_true=user_df['act'].values
            y_pred=user_df.iloc[:,i+2].values
            k = k if k<=(len(y_true)-1) else (len(y_true)-1)
            # print(userid,i,k,y_pred)
            y_IMDB=imdb_df.loc[imdb_df['itemid'].isin(user_df['itemid']), 'Rating'].values
            # score.append([userid, algo, k, y_true, y_pred, y_IMDB])
            result1 = ndcg_mrr_fpc_k(y_true, y_pred, y_IMDB, k)
            result2 = MAE_MSE_RMSE_R2_at_K(y_true, y_pred, k)
            algo=('ItemKNN') if (i==0) else ('UserKNN') if(i==1) else ('SVD') if (i==2) else ('SVD++') if (i==3) else ('Random') if(i==4) else ('algo')
            # score.append([userid, algo, k, list(result1+result2)])
            score.append([userid, algo, k])
            score[len(score)-1].extend(result1)
            score[len(score)-1].extend(result2)

C:\Users\bhask\AppData\Local\Temp\ipykernel_2788\711803717.py:124: RuntimeWarning: divide by zero encountered in scalar divide
  R2 = 1 - (SS_res/SS_tot)


In [ ]:
score_df=pd.DataFrame(score)
score_df.to_csv('Eval_Results.csv')
print(score_df)

            0           1   2         3               4         5         6  \
0      userid  Model Name   K    ndcg@k  Relative_mrr@k     fcp@k     MAE@k   
1           1     ItemKNN   3  0.878896        0.611111       1.0   0.46763   
2           1     ItemKNN   5  0.912475        0.456667       0.8  0.656766   
3           1     ItemKNN  10  0.757806        0.292897  0.888889  0.725461   
4           1     ItemKNN  15  0.695399        0.221215  0.809524  0.687333   
...       ...         ...  ..       ...             ...       ...       ...   
23371     943      Random   3  0.788068        0.611111       1.0  0.941616   
23372     943      Random   5  0.846831        0.456667       0.8  0.700536   
23373     943      Random  10  0.740618        0.292897  0.822222  0.960565   
23374     943      Random  15  0.773904        0.221215  0.695238  0.892478   
23375     943      Random  20  0.769444        0.179887  0.715789  0.850105   

              7         8          9  
0         MS

In [ ]:
score_df=pd.DataFrame(score)
order=['userid', 'Model Name', 'K', 'ndcg@k', 'Relative_mrr@k','fcp@k', 'MAE@k', 'MSE@k', 'RMSE@k', 'R2squared']
score_df.columns= order
score_df = score_df[score_df['userid'] != 'userid']
score_df.replace([np.inf, -np.inf], 0, inplace=True)
score_df

,userid,Model Name,K,ndcg@k,Relative_mrr@k,fcp@k,MAE@k,MSE@k,RMSE@k,R2squared
1,1,ItemKNN,3,0.878896,0.611111,1.000000,0.467630,0.333360,0.577373,0.092094
2,1,ItemKNN,5,0.912475,0.456667,0.800000,0.656766,0.553865,0.744221,0.092094
3,1,ItemKNN,10,0.757806,0.292897,0.888889,0.725461,0.832181,0.912240,0.092094
4,1,ItemKNN,15,0.695399,0.221215,0.809524,0.687333,0.737701,0.858895,0.092094
5,1,ItemKNN,20,0.711270,0.179887,0.736842,0.930960,1.480289,1.216671,0.092094
...,...,...,...,...,...,...,...,...,...,...
23371,943,Random,3,0.788068,0.611111,1.000000,0.941616,2.659924,1.630927,-0.063713
23372,943,Random,5,0.846831,0.456667,0.800000,0.700536,1.642919,1.281764,-0.063713
23373,943,Random,10,0.740618,0.292897,0.822222,0.960565,2.075121,1.440528,-0.063713
23374,943,Random,15,0.773904,0.221215,0.695238,0.892478,1.653753,1.285983,-0.063713


In [ ]:
# calculate the average value over the dataset. for a given model and given k
# score=[['userid', 'Model Name', 'K', 'ndcg@k', 'Relative_mrr@k','fcp@k', 'MAE@k', 'MSE@k', 'RMSE@k', 'R2squared']]
Avgscore_df = score_df.groupby(['Model Name', 'K']).mean().reset_index()
Avgscore_df.replace([np.inf, -np.inf], 0, inplace=True)
Avgscore_df.to_csv('AvgEval_Results.csv')
Avgscore_df

,Model Name,K,userid,ndcg@k,Relative_mrr@k,fcp@k,MAE@k,MSE@k,RMSE@k,R2squared
0,ItemKNN,3,470.664213,0.666807,0.577635,0.651947,0.764572,0.909986,0.874424,-0.009953
1,ItemKNN,4,434.871287,0.775456,0.461015,0.632013,0.781569,0.962105,0.924249,-0.081717
2,ItemKNN,5,474.380857,0.702806,0.434972,0.675773,0.756953,0.911037,0.897630,0.007653
3,ItemKNN,6,544.830189,0.835816,0.363208,0.705660,0.787884,0.977239,0.940145,-0.025955
4,ItemKNN,7,522.461538,0.769780,0.339508,0.622711,0.763233,0.925349,0.920975,-0.022573
...,...,...,...,...,...,...,...,...,...,...
85,UserKNN,16,569.083333,0.800876,0.204191,0.757639,0.806289,1.117471,1.021111,-0.055236
86,UserKNN,17,518.444444,0.830382,0.198064,0.703431,0.822915,0.982052,0.973179,-0.011023
87,UserKNN,18,377.083333,0.850553,0.189365,0.758170,0.742380,0.827879,0.895589,-0.031367
88,UserKNN,19,480.916667,0.873207,0.180827,0.739766,0.807357,1.027321,0.979028,-0.049629


In [ ]:
ModelAvgScore_df=score_df.groupby(['Model Name']).mean().reset_index()
ModelAvgScore_df.replace([np.inf, -np.inf], 0, inplace=True)
ModelAvgScore_df.to_csv('ModelAvgEval_Results.csv')
ModelAvgScore_df

,Model Name,userid,K,ndcg@k,Relative_mrr@k,fcp@k,MAE@k,MSE@k,RMSE@k,R2squared
0,ItemKNN,470.921925,7.905455,0.723901,0.388017,0.677404,0.765064,0.925258,0.911331,0.003140
1,Random,470.921925,7.905455,0.720769,0.393574,0.673480,1.009066,1.753438,1.230465,-1.430426
2,SVD,470.921925,7.905455,0.723804,0.399950,0.677211,0.772235,0.990545,0.932613,-0.074734
3,SVD++,470.921925,7.905455,0.724187,0.399964,0.677531,0.770876,1.001274,0.935450,-0.066153
4,UserKNN,470.921925,7.905455,0.723914,0.403887,0.677441,0.843759,1.149343,1.004770,-0.413223
